In [1]:
#import packages
from datetime import datetime
import geopandas as gp
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import descartes
from shapely.geometry import Point
import descartes
import geoplot as gplt
import geoplot.crs as gcrs
import mpld3
from bokeh.io import output_notebook, show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, Slider, HoverTool
from bokeh.palettes import brewer
from bokeh.layouts import widgetbox, row, column

In [3]:
# code to get data from Ookla for MA and seperated by cities
def quarter_start(year: int, q: int) -> datetime:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")

    month = [1, 4, 7, 10]
    return datetime(year, month[q - 1], 1)


def get_tile_url(service_type: str, year: int, q: int) -> str:
    dt = quarter_start(year, q)

    base_url = "https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance"
    url = f"{base_url}/type%3D{service_type}/year%3D{dt:%Y}/quarter%3D{q}/{dt:%Y-%m-%d}_performance_fixed_tiles.zip"
    return url

# county_url = "https://www2.census.gov/geo/tiger/TIGER2019/COUNTY/tl_2019_us_county.zip" 
# counties = gp.read_file(county_url)
# ma_counties = counties.loc[counties['STATEFP'] == '25'].to_crs(epsg=4326)


ma_cities = gp.read_file('./data/export-gisdata.mapc.ma_municipalities').to_crs(epsg=4326)
# shapefiles from datacommon

def get_data_to_csv(service_type: str, year: int, q: int):
    #read data
    print(f'start processing quarter{q}')
    tile_url = get_tile_url(service_type, year, q)
    tiles = gp.read_file(tile_url)
    
    #get data for MA
    print('start find data for MA')
    tiles_in_ma_cities = gp.sjoin(tiles, ma_cities, how="inner", op='intersects')
    
    #convert to csv
    print('converting')
    tiles_in_ma_cities.to_csv(path_or_buf= f'./ookla2020q{q}ma_bycity.csv')
    return None

def get_data_to_shp(service_type: str, year: int, q: int):
    #read data
    print(f'start processing quarter{q}')
    tile_url = get_tile_url(service_type, year, q)
    tiles = gp.read_file(tile_url)
    
    #get data for MA
    print('start find data for MA')
    tiles_in_ma_cities = gp.sjoin(tiles, ma_cities, how="inner", op='intersects')
    
    #convert to csv
    print('converting')
    tiles_in_ma_cities.to_file(filename= f'./Ookla shape data/{year}q{q}/ookla2020q{q}ma_bycity.shp')
    return None

In [ ]:
# plot the data
q1 = gp.read_file('./Ookla shape data/2020q1')
q2 = gp.read_file('./Ookla shape data/2020q2')
q3 = gp.read_file('./Ookla shape data/2020q3')
q4 = gp.read_file('./Ookla shape data/2020q4')
ma = gp.read_file('./data/export-gisdata.mapc.ma_municipalities').to_crs(epsg=4326)

# convert from kbps to mbps
ma_city_plot_data['avg_d_mbps'] = ma_city_plot_data['avg_d_kbps'] / 1000
ma_city_plot_data['avg_u_mbps'] = ma_city_plot_data['avg_u_kbps'] / 1000

# plot coverage
ax = gplt.webmap(data_2020, projection=gcrs.WebMercator())
gplt.kdeplot(data_2020[['avg_d_kbps', 'geometry']], n_levels=50, cmap='Greens', thresh=0.05 ,shade=True, ax=ax)

# plot download speed
ax = gplt.webmap(data_2020, projection=gcrs.WebMercator())
gplt.choropleth(ma_city_plot_data, hue='avg_d_mbps', projection=gcrs.AlbersEqualArea(),
    cmap='Greens', legend=True, ax=ax)
plt.title('Average Download Speed in Mbps')

# plot upload speed
ax = gplt.webmap(data_2020, projection=gcrs.WebMercator())
fig = gplt.choropleth(ma_city_plot_data, hue='avg_u_mbps', projection=gcrs.AlbersEqualArea(),
    cmap='Reds', legend=True, ax=ax)
plt.title('Average Upload Speed in Mbps')



In [ ]:
# plot interactive data

# modified from https://towardsdatascience.com/a-complete-guide-to-an-interactive-geographical-map-using-python-f4c5197e23e0
import json
#Read data to json.
merged_json = json.loads(ma_city_plot_data.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)
#Define a sequential multi-hue color palette.
palette = brewer['YlGn'][9]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 200)
#Define custom tick labels for color bar.
tick_labels = {'0': '0 Mbps', '50':'50 Mbps', '100':'100 Mbps', '150':'150 Mbps', '200':'200 Mbps'}
#Add hover tool
hover = HoverTool(tooltips = [ ('City','@municipal'),('Average Download Speed', '@avg_d_mbps Mbps'),
                              ('Average Upload Speed', '@avg_u_mbps Mbps'), ('Average Number of Tests', '@tests')])
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width = 500, height = 20,
border_line_color=None, location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = 'MA Boradband Download Distribution, 2020', plot_height = 600 , plot_width = 950,
           toolbar_location = None, tools = [hover])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'avg_d_mbps', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')
#Add label
p.xaxis.axis_label = "Longtitude"
p.yaxis.axis_label = "Latitude"
#Display figure inline in Jupyter Notebook.
output_notebook()
output_file("MA.html")
#Display figure
show(p)